In [ ]:
import sys
sys.path.append("ALL_IN_ONE/Pythia8")
sys.path.append("ALL_IN_ONE/LSD")
sys.path.append("ALL_IN_ONE/FIT_FUNC")
import os
import ALL_IN_ONE.Pythia8.product_test as pt
import ALL_IN_ONE.Pythia8.functions_for_run as fr
from numba import jit
import ALL_IN_ONE.Pythia8.cross_section as cs
from tqdm import tqdm
import pandas as pd
import ALL_IN_ONE.Pythia8.run_save as rs
import ALL_IN_ONE.Pythia8.one_key_run as okr
import ALL_IN_ONE.Pythia8.combine as cb


## TEST of SHiP-Detection func

In [ ]:

@jit(nopython=True)

def add_whether_in_the_detector_without_Decay_calcu_add_cross_section_SHiP(filename, out_folder_path):
    rs.mkdir_1(out_folder_path)
    file_path_only, file_name_only = os.path.split(filename)
    file_parent_path_only = os.path.dirname(file_path_only)
    llp_data = pd.read_csv(filename)
    llp_whether_in_detector_SHiP = llp_data.apply(
        lambda row: pt.is_point_inside_frustum_numba_array([row['decay_pos_x'], row['decay_pos_y'], row['decay_pos_z']]),
        axis=1
    )


    cross_section = cs.calculate_cross_section(llp_data)

    llp_data['detected_SHiP'] = llp_whether_in_detector_SHiP
    llp_data['cross_section'] = cross_section
    llp_data['detector_acceptance_SHiP'] = sum(llp_data['detected_SHiP']) / cs.counting_total_LLP(llp_data)
    final_data_folder = file_parent_path_only + '/Completed_llp_data_precise_cross_section'
    fr.mkdir_1(final_data_folder)
    final_data_path = os.path.join(final_data_folder + f'/final_data_cross_section_{file_name_only}')
    llp_data.to_csv(final_data_path, index = False)
    return final_data_folder

def SHiP(LLP_data_folder_dir):
    for files in tqdm(os.listdir(LLP_data_folder_dir)):
        file_path_all = os.path.join(LLP_data_folder_dir, files)
        if os.path.isfile(file_path_all):
            try:
                completed_data_folder = add_whether_in_the_detector_without_Decay_calcu_add_cross_section_SHiP(file_path_all, LLP_data_folder_dir)
            except Exception as e:
                print(f"Error with file: {file_path_all}")
                print(f"Error message: {str(e)}")
                continue
                file_path_all = ''

    return 'Detection and Calcu Cross-Section Completed', completed_data_folder



In [4]:
folder = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/SHiP'
SHiP(folder)

100%|██████████| 1/1 [00:00<00:00, 77.87it/s]

Error with file: /media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/SHiP/test.csv
Error message: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'mkdir_1' of type Module(<module 'ALL_IN_ONE.Pythia8.run_save' from '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Program/PRA/Github/position_read_analyse_1.3/ALL_IN_ONE/Pythia8/run_save.py'>)

File "../../../../../../../tmp/ipykernel_83791/3487130479.py", line 17:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_83791/3487130479.py (17)

File "../../../../../../../tmp/ipykernel_83791/3487130479.py", line 17:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference


UnboundLocalError: local variable 'completed_data_folder' referenced before assignment

In [2]:
import pandas as pd
csv = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/SHiP/test.csv'
df = pd.read_csv(csv)
df2 = df[df['detected_SHiP'] == 1]
print(df2[:5])

Empty DataFrame
Columns: [number_of_production, m, id, total_events, seed, decay_pos_x, LLP_number_per_ev, Cross_section_fb, decay_pos_y, decay_pos_t, mass_input, tau, theta, decay_pos_z, tau_input, detected_SHiP, cross_section, detector_acceptance_SHiP]
Index: []


## Move CODEXb Around

In [4]:
detector_xmin=26000
detector_xmax=36000
detector_ymin=-7000
detector_ymax=3000
detector_zmin=5000
detector_zmax=15000

center = [(detector_xmin + detector_xmax) / 2
                , (detector_ymin + detector_ymax) / 2
                , (detector_zmin + detector_zmax) / 2]
xplus = (detector_xmax - detector_xmin) / 2
yplus = (detector_ymax - detector_ymin) / 2
zplus = (detector_zmax - detector_zmin) / 2

print(center)
print(xplus, yplus, zplus)
print(center[0]+xplus, center[0]-xplus)

[31000.0, -2000.0, 10000.0]
5000.0 5000.0 5000.0
36000.0 26000.0


In [ ]:
folder = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/CODEX-b'
out = '/media/ubuntu/6156e08b-fdb1-4cde-964e-431f74a6078e/Files/LLP_DATA/Original_DATA/test_data/CODEX-b/out'
okr.CODEXb_bigger(folder, out)